# Projects using Pytorch

**Outline**

- [Project one - predicting the fuel efficiency of a car](#Project-one----predicting-the-fuel-efficiency-of-a-car)
  - [Working with feature columns](#Working-with-feature-columns)
  - [Training a DNN regression model](#Training-a-DNN-regression-model)
- [Project two - classifying MNIST handwritten digits](#Project-two----classifying-MNIST-handwritten-digits)

In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

## Project one - predicting the fuel efficiency of a car


We will work
on a real-world project of predicting the fuel efficiency of a car in miles per gallon (MPG).

### Working with feature columns
- In machine learning and deep learning applications, we can encounter various different types of features: continuous, unordered categorical (nominal), and ordered categorical (ordinal).

- while numeric data can be either continuous
or discrete, in the context of machine learning with PyTorch, “numeric” data specifically refers to
continuous data of floating point type.


The features (model year, cylinders, displacement, horsepower, weight, acceler-
ation, and origin) were obtained from the Auto MPG dataset, which is a common machine learning
benchmark dataset for predicting the fuel efficiency of a car in MPG. The full dataset and its description are available from UCI’s machine learning repository at https://archive.ics.uci.edu/ml/
datasets/auto+mpg.

---

We are going to treat five features from the Auto MPG dataset (number of cylinders, displacement,
horsepower, weight, and acceleration) as “numeric” (here, continuous) features. The model year
can be regarded as an ordered categorical (ordinal) feature. Lastly, the manufacturing origin can be
regarded as an unordered categorical (nominal) feature with three possible discrete values, 1, 2, and
3, which correspond to the US, Europe, and Japan, respectively.

In [2]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

df = pd.read_csv(url, names=column_names,
                 na_values = "?", comment='\t',
                 sep=" ", skipinitialspace=True)

df.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [3]:
print(df.isna().sum())

df = df.dropna()
df = df.reset_index(drop=True)
df.tail()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
387,27.0,4,140.0,86.0,2790.0,15.6,82,1
388,44.0,4,97.0,52.0,2130.0,24.6,82,2
389,32.0,4,135.0,84.0,2295.0,11.6,82,1
390,28.0,4,120.0,79.0,2625.0,18.6,82,1
391,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [4]:
import sklearn
import sklearn.model_selection


df_train, df_test = sklearn.model_selection.train_test_split(df, train_size=0.8, random_state=1)
train_stats = df_train.describe().transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
MPG,313.0,23.404153,7.666909,9.0,17.5,23.0,29.0,46.6
Cylinders,313.0,5.402556,1.701506,3.0,4.0,4.0,8.0,8.0
Displacement,313.0,189.512780,102.675646,68.0,104.0,140.0,260.0,455.0
Horsepower,313.0,102.929712,37.919046,46.0,75.0,92.0,120.0,230.0
Weight,313.0,2961.198083,848.602146,1613.0,2219.0,2755.0,3574.0,5140.0
Acceleration,313.0,15.704473,2.725399,8.5,14.0,15.5,17.3,24.8
Model Year,313.0,75.929712,3.675305,70.0,73.0,76.0,79.0,82.0
Origin,313.0,1.591054,0.807923,1.0,1.0,1.0,2.0,3.0


In [5]:
from packaging import version


numeric_column_names = ['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']

df_train_norm, df_test_norm = df_train.copy(), df_test.copy()


for col_name in numeric_column_names:
        mean = train_stats.loc[col_name, 'mean']
        std = train_stats.loc[col_name, 'std']
        df_train_norm[col_name] = (df_train_norm[col_name] - mean) / std
        df_test_norm[col_name] = (df_test_norm[col_name] - mean) / std

df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
203,28.0,-0.824303,-0.901020,-0.736562,-0.950031,0.255202,76,3
255,19.4,0.351127,0.413800,-0.340982,0.293190,0.548737,78,1
72,13.0,1.526556,1.144256,0.713897,1.339617,-0.625403,72,1
235,30.5,-0.824303,-0.891280,-1.053025,-1.072585,0.475353,77,1
37,14.0,1.526556,1.563051,1.636916,1.470420,-1.359240,71,1


let’s group the rather fine-grained model year (ModelYear) information into buckets to simplify
the learning task for the model that we are going to train later.

In
order to group the cars into these buckets, we will first define three cut-off values: [73, 76, 79] for the
model year feature. These cut-off values are used to specify half-closed intervals, for instance, (–∞, 73),
[73, 76), [76, 79), and [76, ∞). Then, the original numeric features will be passed to the torch.bucketize
function (https://pytorch.org/docs/stable/generated/torch.bucketize.html) to generate the
indices of the buckets.

In [6]:
boundaries = torch.tensor([73, 76, 79])

v = torch.tensor(df_train_norm['Model Year'].values)
df_train_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)

v = torch.tensor(df_test_norm['Model Year'].values)
df_test_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)

numeric_column_names.append('Model Year Bucketed')

Next, we will proceed with defining a list for the `unordered categorical feature`, `Origin`.
- In PyTorch,
There are two ways to work with a categorical feature: using an embedding layer via nn.Embedding
(https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html), or using one-hot-encoded vectors (also called indicator).
- In the encoding approach, for example, index 0 will be encoded
as [1, 0, 0], index 1 will be encoded as [0, 1, 0], and so on.
- On the other hand, the embedding layer
maps each index to a vector of random numbers of the type float, which can be trained.
- We can
think of the embedding layer as a more efficient implementation of a one-hot encoding multiplied
with a trainable weight matrix.
- When the number of categories is large, using the embedding layer with fewer dimensions than the
number of categories can improve the performance.

In [7]:
from torch.nn.functional import one_hot


total_origin = len(set(df_train_norm['Origin']))

origin_encoded = one_hot(torch.from_numpy(df_train_norm['Origin'].values) % total_origin)
x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)
x_train = torch.cat([x_train_numeric, origin_encoded], 1).float()

origin_encoded = one_hot(torch.from_numpy(df_test_norm['Origin'].values) % total_origin)
x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)
x_test = torch.cat([x_test_numeric, origin_encoded], 1).float()


Finally, we will create the `label tensors` from
the `ground truth MPG values`

In [8]:
y_train = torch.tensor(df_train_norm['MPG'].values).float()
y_test = torch.tensor(df_test_norm['MPG'].values).float()

### Training a DNN regression model

Let's create a data loader that uses a
`batch size of 8` for the train data

In [9]:
from torch.utils.data import DataLoader, TensorDataset


train_ds = TensorDataset(x_train, y_train)
batch_size = 8
torch.manual_seed(1)
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

Next, let's build a model with `two fully connected layers` where one has `8 hidden units` and another
has `4`

In [10]:
hidden_units = [8, 4]
input_size = x_train.shape[1]

all_layers = []
for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1], 1))

model = nn.Sequential(*all_layers)

model

Sequential(
  (0): Linear(in_features=9, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
)

We will use the `MSE` loss function for regression and `stochastic gradient descent` for optimization

In [11]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

Now let's will train the model for `200 epochs` and display the train loss for every 20 epochs

In [12]:
torch.manual_seed(1)
num_epochs = 200
log_epochs = 20
for epoch in range(num_epochs):
    loss_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)[:, 0]
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_hist_train += loss.item()
    if epoch % log_epochs==0:
        print(f'Epoch {epoch}  Loss {loss_hist_train/len(train_dl):.4f}')


Epoch 0  Loss 536.1047
Epoch 20  Loss 8.4361
Epoch 40  Loss 7.8695
Epoch 60  Loss 7.1891
Epoch 80  Loss 6.7062
Epoch 100  Loss 6.7599
Epoch 120  Loss 6.3124
Epoch 140  Loss 6.6864
Epoch 160  Loss 6.7648
Epoch 180  Loss 6.2156


We can now evaluate the regression performance of the
trained model on the test dataset

In [13]:
with torch.no_grad():
    pred = model(x_test.float())[:, 0]
    loss = loss_fn(pred, y_test)
    print(f'Test MSE: {loss.item():.4f}')
    print(f'Test MAE: {nn.L1Loss()(pred, y_test).item():.4f}')

Test MSE: 9.6133
Test MAE: 2.1211


The `MSE` on the test set is `9.6`, and the mean absolute error (`MAE`) is `2.1`

## Project two - classifying MNIST hand-written digits

The setup step includes loading the dataset and specifying hyperparameters (the size of the
train set and test set, and the size of mini-batches)

Here, we will construct a data loader with `batches of 64 samples`.

In [14]:
import torchvision
from torchvision import transforms


image_path = './'
transform = transforms.Compose([transforms.ToTensor()])

mnist_train_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=True,
                                           transform=transform,
                                           download=True)
mnist_test_dataset = torchvision.datasets.MNIST(root=image_path,
                                           train=False,
                                           transform=transform,
                                           download=False)

batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size, shuffle=True)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.04MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 135kB/s]
100%|██████████| 1.65M/1.65M [00:06<00:00, 241kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.78MB/s]


- We preprocessed the input features and the labels. The features in this project are the pixels of the images.
of the images.
- We defined a custom transformation using `torchvision.transforms.Compose`.
- In this simple case, our transformation consisted only of one method, `ToTensor()`.
- The ToTensor() method converts the pixel features into a floating type tensor
and also normalizes the pixels from the [0, 255] to [0, 1] range.
- The labels are integers from 0 to
9 representing ten digits. Hence, we don’t need to do any scaling or further conversion.

Now let's construct the NN model

- The model starts with a flatten layer that flattens an input image into
a one-dimensional tensor. This is because the input images are in the shape of
`[1, 28, 28]`.
- The model has `two hidden layers, with 32 and 16 units` respectively.
And it ends with an `output layer of 10 units` representing ten classes, `activated by a softmax function`.

In [15]:
hidden_units = [32, 16]
image_size = mnist_train_dataset[0][0].shape
input_size = image_size[0] * image_size[1] * image_size[2]

all_layers = [nn.Flatten()]
for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1], 10))
model = nn.Sequential(*all_layers)

model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
)

let's use the model for training, evaluation, and prediction

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

torch.manual_seed(1)
num_epochs = 20
for epoch in range(num_epochs):
    accuracy_hist_train = 0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
        accuracy_hist_train += is_correct.sum()
    accuracy_hist_train /= len(train_dl.dataset)
    print(f'Epoch {epoch}  Accuracy {accuracy_hist_train:.4f}')

Epoch 0  Accuracy 0.8531
Epoch 1  Accuracy 0.9287
Epoch 2  Accuracy 0.9413
Epoch 3  Accuracy 0.9506
Epoch 4  Accuracy 0.9558
Epoch 5  Accuracy 0.9592
Epoch 6  Accuracy 0.9627
Epoch 7  Accuracy 0.9649
Epoch 8  Accuracy 0.9673
Epoch 9  Accuracy 0.9690
Epoch 10  Accuracy 0.9711
Epoch 11  Accuracy 0.9729
Epoch 12  Accuracy 0.9737
Epoch 13  Accuracy 0.9747
Epoch 14  Accuracy 0.9766
Epoch 15  Accuracy 0.9778
Epoch 16  Accuracy 0.9780
Epoch 17  Accuracy 0.9798
Epoch 18  Accuracy 0.9806
Epoch 19  Accuracy 0.9816


- We used the `cross-entropy loss function` for multiclass classification and the `Adam optimizer`
for gradient descent.
- We trained the model
for `20 epochs` and displayed the train accuracy for every epoch.

In [17]:
pred = model(mnist_test_dataset.data / 255.)
is_correct = (torch.argmax(pred, dim=1) == mnist_test_dataset.targets).float()
print(f'Test accuracy: {is_correct.mean():.4f}')


Test accuracy: 0.9645


---

